<a href="https://colab.research.google.com/github/SimoRada/EmployeeDetection/blob/main/ModelTraining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INTRODUCTION**

I have developed this code to train a machine learning model called Mobilnet-V2, using a dataset in COCO format, and then export it to a *.tflite* file to use in the final code.

# SETUP

This section presents the Python libraries that the code imports in order to use the model retraining APIs and extract in TensorFlow Lite format.

Use the following code to import the required Python classes:

In [ ]:
import os
import json
import tensorflow as tf
assert tf.__version__.startswith('2')
from mediapipe_model_maker import object_detector

# Prepare data

Our directory "employee" contains two subdirectories:
both of them contains a set of photos inside the folder "images" and a JSON file that describes the coordinates associated with each image, wich the computer must interpret to determibe what to focus in the photo.

*   *employee/train* is used to train the model
*   *employee/validation* is used to test the model

Then I save that in the respective variables: *train_dataset_path* and *validation_dataset_path*.

In [ ]:
train_dataset_path = "employee/train"
validation_dataset_path = "employee/validation"

#Create dataset


Now we use the *from_coco_folder* method to load the dataset located at *train_dataset_path* and *validation_dataset_path*.
During dataset loading, the data will be read from the specified path and converted into a uniform TFRecord format, which is stored in a cache for future use. It’s important to set up a cache_dir and consistently use it throughout your training to avoid creating redundant caches of the same dataset.

In [ ]:
train_data = object_detector.Dataset.from_coco_folder(train_dataset_path, cache_dir="/tmp/od_data/train")
validation_data = object_detector.Dataset.from_coco_folder(validation_dataset_path, cache_dir="/tmp/od_data/validation")

# Set retraining options

In addition to the training dataset, there are a few essential settings needed to run retraining: the model's output directory and its architecture. To define the output directory, use the *HParams* class to set the *export_dir* parameter. To specify the model architecture, use the *SupportedModels* class. The object detection solution supports the following model architectures:


*   MobileNet-V2
*   MobileNet-MultiHW-AVG

Also in this section we can modify the epochs (look "epochs = 15")
One epoch is one complete pass of the training data set through the algorithm


In [ ]:
spec = object_detector.SupportedModels.MOBILENET_MULTI_AVG
hparams = object_detector.HParams(epochs = 15, export_dir='exported_model')
options = object_detector.ObjectDetectorOptions(
    supported_model=spec,
    hparams=hparams
)

# Run retraining

Once your training dataset and retraining options are set up, you can begin the retraining process. This task is resource-demanding and may take anywhere from a few minutes to several hours, depending on your available compute resources.
To initiate the retraining process, call the create() method with the dataset and options configured earlier:

In [ ]:
model = object_detector.ObjectDetector.create(
    train_data=train_data,
    validation_data=validation_data,
    options=options)